# Plotly PNG Export — Robust Example Notebook

In [ ]:
# Install/upgrade kaleido once (safe to re-run)
try:
    import kaleido  # noqa: F401
except Exception:
    %pip -q install -U kaleido


In [ ]:
import importlib
import plotly.express as px
import plotly.io as pio
import pandas as pd

def save_plotly_png(fig, filename):
    if importlib.util.find_spec("kaleido") is None:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "kaleido"])
    try:
        try:
            pio.kaleido.scope.default_format = "png"
        except Exception:
            pass
        fig.write_image(filename, engine="kaleido")
        print(f"✅ PNG exported: {filename}")
    except Exception as e:
        html_name = filename.replace(".png", ".html")
        pio.write_html(fig, html_name, include_plotlyjs="cdn", full_html=True)
        print(f"⚠️ PNG export failed ({e}). Saved HTML instead: {html_name}")


In [ ]:
gm = px.data.gapminder().query("year == 2007").copy()
happy_df = pd.DataFrame({
    "Country or region": gm["country"],
    "GDP per capita": gm["gdpPercap"],
    "Score": gm["lifeExp"],
    "Overall rank": gm["pop"].rank(ascending=False).astype(int)
})
happy_df.head()


In [ ]:
fig = px.scatter(
    happy_df,
    x="GDP per capita",
    y="Score",
    size="Overall rank",
    color="Country or region",
    hover_name="Country or region",
    title="Happiness Score vs GDP per Capita"
)

target = "Taiwan" if (happy_df["Country or region"] == "Taiwan").any() else ("Japan" if (happy_df["Country or region"] == "Japan").any() else happy_df.iloc[0]["Country or region"])
row = happy_df.loc[happy_df["Country or region"] == target].iloc[0]
fig.add_annotation(x=row["GDP per capita"], y=row["Score"], text=target, showarrow=True, arrowhead=2, ax=20, ay=-40)

fig.show()
save_plotly_png(fig, "fig1.png")


![fig1](fig1.png)

In [ ]:
corr = happy_df.select_dtypes("number").corr()
fig = px.imshow(corr, text_auto=True, aspect="auto", title="Correlation heatmap")
fig.update_layout(margin=dict(l=40, r=20, t=40, b=40))
fig.show()
save_plotly_png(fig, "fig2.png")


![fig2](fig2.png)

In [ ]:
fig = px.scatter_matrix(
    happy_df,
    dimensions=["GDP per capita", "Score", "Overall rank"],
    color="Country or region",
    title="Scatter Matrix"
)
fig.update_traces(diagonal_visible=False)
fig.show()
save_plotly_png(fig, "fig3.png")


![fig3](fig3.png)